In [1]:
import pandas as pd
from seeq import spy
import livef1

In [2]:
# Constants

# Workbook name will scope all the signals to a particular workbook, Change to None if you want them 
# globally soped and available in any workbook 

WORKBOOK_NAME = 'F1 Telemetry Testing'
DATASOURCE_NAME = 'F1 Telemetry Playground'
SESSION_CONDITION_NAME = 'F1 Sessions'
CURRENT_SEASON = 2025


In [3]:
def add_capsule_start_end_session_df (df):
    df['session_startDate'] = pd.to_datetime(df['session_startDate'])
    df['gmtoffset'] = pd.to_timedelta(df['gmtoffset'])
    df['Capsule Start'] = (df['session_startDate'] + df['gmtoffset'])
    df['Capsule End'] = (df['session_endDate'] + df['gmtoffset'])
    df['Capsule Start'] = pd.to_datetime(df['Capsule Start'], utc=True)
    df['Capsule End'] = pd.to_datetime(df['Capsule End'], utc=True)
    return df

def set_signals_max_interpolation():
    scoped_workbook_id = None
    if WORKBOOK_NAME != None:
        scoped_workbook = spy.search({'Name':WORKBOOK_NAME, 'Type':'Workbook'})
        scoped_workbook_id = scoped_workbook['ID'][0]
    
    telemetry_signals = spy.search({'Name':'20??.*.*.*','Type':'Signal','Scoped To':scoped_workbook_id})
    metadata = telemetry_signals.copy()
    metadata['Maximum Interpolation'] = '5 min'
    # Push the updated metadata back to Seeq
    spy.push(metadata=metadata)
    


def get_session_data(current_time):
    """Fetch session data from Seeq."""
    scoped_workbook_id = None
    if WORKBOOK_NAME != None:
        scoped_workbook = spy.search({'Name':WORKBOOK_NAME, 'Type':'Workbook'})
        scoped_workbook_id = scoped_workbook['ID'][0]
        
    posted_session_condition = spy.search({'Name': SESSION_CONDITION_NAME, 'Datasource Name': DATASOURCE_NAME, 'Scoped To':scoped_workbook_id})
    posted_sessions = spy.pull(posted_session_condition, start='2023-01-01T00:00:00Z', end=current_time)
    return posted_sessions[posted_sessions['Capsule End'] < current_time]

def filter_sessions(sessions_df, posted_sessions):
    """Filter sessions that have not been processed yet."""
    return sessions_df[~sessions_df['session_key'].isin(posted_sessions['session_key'])]

def process_telemetry_data(telemetry_data, current_season, current_team_name, current_driver_no):
    """Process telemetry data for a specific driver."""
    telemetry_data = telemetry_data.dropna(subset=['DriverNo'])
    telemetry_data['DriverNo'] = pd.to_numeric(telemetry_data['DriverNo'], errors='coerce').astype('Int64')
    telemetry_data = telemetry_data.drop(columns=['SessionKey', 'DriverNo', 'timestamp'])
    telemetry_data['Utc'] = pd.to_datetime(telemetry_data['Utc'])
    telemetry_data = telemetry_data.set_index('Utc')

    # Rename columns
    telemetry_data.columns = [
        f"{current_season}.{current_team_name}.{current_driver_no}.{col}" for col in telemetry_data.columns
    ]
    return telemetry_data

def process_laps_data(laps_data, current_team_name, current_driver_name):
    """Process laps data for a specific driver."""
    laps_data = laps_data.dropna(subset=['lap_time'])
    laps_data['team_name'] = current_team_name
    laps_data['current_driver_name'] = current_driver_name
    return laps_data

def create_sector_capsules(laps_data, sector_number):
    """Create sector capsules for a specific sector."""
    sector_caps = laps_data.copy()
    
    if sector_number == 1:
        sector_caps['Capsule Start'] = sector_caps['lap_start_date']
        sector_caps['Capsule End'] = sector_caps['Capsule Start'] + sector_caps['sector1_time']
    if sector_number == 2:
        sector_caps['Capsule Start'] = sector_caps['lap_start_date'] + sector_caps['sector1_time']
        sector_caps['Capsule End'] = sector_caps['Capsule Start'] + sector_caps['sector2_time']
    if sector_number == 3:
        sector_caps['Capsule Start'] = sector_caps['lap_start_date']+sector_caps['sector1_time']+sector_caps['sector2_time']
        sector_caps['Capsule End'] = sector_caps['Capsule Start'] + sector_caps['sector3_time']
    
    sector_caps['Sector'] = sector_number
    sector_caps = sector_caps[['lap_number', 'in_pit', 'DriverNo', 'Capsule Start', 'Capsule End', 'Sector', 'team_name', 'current_driver_name']]
    return sector_caps.dropna(subset=['Capsule Start', 'Capsule End'])

def push_telemetry_data(telemetry_data):
    """Push telemetry data to Seeq."""
    spy.push(data=telemetry_data, workbook=WORKBOOK_NAME, datasource=DATASOURCE_NAME)

def push_sector_data(all_sectors, current_season, current_team_name, current_driver_no):
    """Push sector data to Seeq."""
    metadata = pd.DataFrame([{
        'Name': f"{current_season}.{current_team_name}.{current_driver_no}.Sectors",
        'Type': 'Condition',
        'Maximum Duration': '5 min',
    }])
    spy.push(data=all_sectors, metadata=metadata, workbook=WORKBOOK_NAME, datasource=DATASOURCE_NAME)

def process_session(session, session_key, car_number_info, current_season):
    """Process a single session."""
    laps_data = session.get_laps()
    laps_data = laps_data.dropna(subset = ['DriverNo'])
    laps_data['DriverNo'] = pd.to_numeric(laps_data['DriverNo'], errors='coerce').astype('Int64')
    
    telemetry_data = session.get_car_telemetry()
    telemetry_data = telemetry_data.dropna(subset = ['DriverNo'])
    telemetry_data['DriverNo'] = pd.to_numeric(telemetry_data['DriverNo'], errors='coerce').astype('Int64')
    


    cars_in_session = telemetry_data.DriverNo.unique()
    cars_pushed = 0

    current_session_key

    for current_driver_no in cars_in_session:
        try:
            matching_row = car_number_info[
                (car_number_info['MinSessionKey'] <= session_key) &
                (car_number_info['MaxSessionKey'] >= session_key) &
                (car_number_info['DriverNo'] == current_driver_no)
            ]
    
            current_team_name = matching_row['TeamName'].values[0] if not matching_row.empty else None
            current_driver_name = matching_row['DriverName'].values[0] if not matching_row.empty else None
    
            # Process telemetry data
            driver_telemetry_data = process_telemetry_data(telemetry_data[telemetry_data.DriverNo == current_driver_no], current_season, current_team_name, current_driver_no)
            push_telemetry_data(driver_telemetry_data)
    
            # Process laps data
            driver_laps_data = process_laps_data(laps_data[laps_data.DriverNo == current_driver_no], current_team_name, current_driver_name)
    
            # Create sector capsules
            sector_1_caps = create_sector_capsules(driver_laps_data, 1)
            sector_2_caps = create_sector_capsules(driver_laps_data, 2)
            sector_3_caps = create_sector_capsules(driver_laps_data, 3)
    
            all_sectors = pd.concat([sector_1_caps, sector_2_caps, sector_3_caps], ignore_index=True)
            push_sector_data(all_sectors, current_season, current_team_name, current_driver_no)
    
            cars_pushed += 1
        except:
            print(f"Skipping car {current_driver_no} driver {current_driver_name} due to error: {e}")
        continue

    return cars_pushed



In [4]:
"""Main function to process sessions."""
current_time = pd.Timestamp.now(tz='UTC')

# Load season and car information
season = livef1.get_season(CURRENT_SEASON)
season.parse_sessions()
sessions_df = season.season_table
car_number_info = pd.read_csv("data/CarTeamMap.csv")

In [8]:
try:
    posted_sessions = get_session_data(current_time)
    filtered_sessions = filter_sessions(sessions_df, posted_sessions)

except Exception as e:
    print(f"No prior sessions have been uploaded posting all sessions - error text: {e}")
    filtered_sessions = sessions_df

filtered_sessions = add_capsule_start_end_session_df(filtered_sessions)
filtered_sessions

,ID,Type,Name,Time,Count,Pages,Data Processed,Result
0,0F010C5F-6AC8-EAF0-AB3C-1A6BD5CE532B,StoredCondition,F1 Sessions,00:00:00.03,13,1,832 B,Success


0

In [7]:
for index, row in filtered_sessions.iterrows():
    current_session_key = row['session_key']
    session = livef1.get_session(season=CURRENT_SEASON, meeting_key=index, session_key=current_session_key)
    session_generated = False
    try:
        session.generate(silver=True)
        session_generated = True
    except Exception as e:
        print(f"Skipping session {row['session_key']} due to error: {e}")
        continue
        
    if session_generated:
        
        cars_pushed = process_session(session,current_session_key,car_number_info, CURRENT_SEASON)

        if cars_pushed > 0:
            # Push session metadata

            keep_cols = [
                "Capsule Start", 
                "Capsule End",
                "index", 
                "season_year",
                "meeting_location",
                "meeting_offname",
                "meeting_name",
                "meeting_country_name",
                "meeting_circuit_shortname",
                "session_key",
                "session_type",
                "session_name"]
            
            session_capsule = pd.DataFrame([row]).reset_index()
            session_capsule = session_capsule[keep_cols]
            session_capsule = session_capsule.rename(columns={'index': 'meeting_key'})
            
            spy.push(data=session_capsule, metadata=pd.DataFrame([{
                'Name': SESSION_CONDITION_NAME,
                'Type': 'Condition',
                'Maximum Duration': '7d',
            }]), workbook=WORKBOOK_NAME, datasource=DATASOURCE_NAME)


    

In [10]:
spy.jobs.schedule('Every 1 Hour')

,Schedule,Scheduled,Next Run
0,Every 1 Hour,Every hour,2025-04-03 21:00:00 PDT


,Schedule,Scheduled,Next Run
0,Every 1 Hour,Every hour,2025-04-03 21:00:00 PDT
